In [1]:
from datetime import datetime
import process_graph_db as PGDB
import process_engine as PE
import process_object as PO

##### Define `Source` object types and instances

In [2]:
"""
Source objects are assets of the org
In case of users 
    - source objects types are types of emplyees ex: manager, analyst, floor worker
    - source object instances are individual employees ex: manager_x, analyst_y, floor worker_z
In case of materials
    - source object types are types of materials ex: MacBook Air 2020
    - source object instances are specific computers ex: MacBook Air 2020 with serial xxx-yyyy-zzz 

All objects need Source type and instance
"""
att_sales_employee_src = [
    PO.ObjectAttribute(att_name='username', att_dtype='string', att_required=True),
    PO.ObjectAttribute(att_name='usertype', att_dtype='string', att_required=True)
]
o_sales_employee_src = PO.ProcessObjectType(o_name='sales', o_attributes=att_sales_employee_src)

att_material_macbook_src = [
    PO.ObjectAttribute(att_name='name', att_dtype='string', att_required=True),
    PO.ObjectAttribute(att_name='unit', att_dtype='string', att_required=True),
    PO.ObjectAttribute(att_name='available_qty', att_dtype='float', att_required=True),
]
o_material_macbook_src = PO.ProcessObjectType(o_name='MacBook Air 2020', o_attributes=att_material_macbook_src)

In [3]:
o_user_inst_allan = PO.ProcessObjectInstance(o_type=o_sales_employee_src, attributes={'username': 'Allan', 'usertype': 'A', 'sex': 'M'})
o_user_inst_betty = PO.ProcessObjectInstance(o_type=o_sales_employee_src, attributes={'username': 'Betty', 'usertype': 'A', 'sex': 'F'})

o_material_macbook_src_instance = PO.ProcessObjectInstance(o_type=o_material_macbook_src, attributes={'name': 'MacBook Air 2020', 'unit': 'kg', 'available_qty':100})

##### Define `WIP` object types and instances

In [4]:
"""
WIP (Work In Progress) objects are assets that are crossing through a process
They have a Source object they are adding to or deducting from
In case of materials
    - wip object type has a reference to Source object instance and other attributes that will affect the Source object instance ex: MacBook Air 2020
    - wip object instance has information directly affecting the Source object instance ex: 2 MacBook Air 2020 (reduces 2 from the source instance)

Not all objects need WIP type and instance
"""
att_material_macbook_wip = [
    PO.ObjectAttribute(att_name='src', att_dtype='string', att_required=True),
    PO.ObjectAttribute(att_name='requested_qty', att_dtype='float', att_required=True),
]
att_material_macbook_wip = PO.ProcessObjectType(o_name='MacBook Air 2020', o_attributes=att_material_macbook_wip)

In [5]:
o_material_macbook_wip_instance = PO.ProcessObjectInstance(o_type=att_material_macbook_wip, attributes={'src': o_material_macbook_src_instance, 'requested_qty':11.0})

##### Define Activities

In [8]:
"""
a process activity type provides the required object, owner, and method for the activity
a process activity instance takes the action, records the timestamp, and implements the effect
"""
# start activity
a_start = PE.ProcessActivityType(a_name='start', a_object=None, a_owner=None, a_from=None, operation=None)
a_stop = PE.ProcessActivityType(a_name='stop', a_object=None, a_owner=None, a_from=None, operation=None)

def check_availability(**quantities):
    return quantities['requested_qty'] <= quantities['available_qty']
a_check_availability = PE.ProcessActivityType(a_name='Check Availability', a_object=o_material_macbook_wip_instance, a_owner=o_user_inst_allan, a_from=a_start, operation=check_availability)


a_check_availability_ins = PE.ProcessActivityInstance(a_type=a_check_availability)
a_check_availability_ins.action(
    available_qty = a_check_availability.a_object.att['src'].att['available_qty'], 
    requested_qty = a_check_availability.a_object.att['requested_qty']
)





In [20]:
a_check_availability_ins.a_timestamp

datetime.datetime(2023, 12, 2, 17, 6, 34, 169921)